In [1]:
# python数据分析测试。
# 测试开始前，请运行该代码，初始化程序
####  第一大题 ####
# 该代码将会输出道指30只成分股的最新股价，并保存为 djidf ，这是一个 DataFrame 格式数据。测试从这里开始
import requests
import re
import pandas as pd

def retrieve_dji_list():
    try:
        r = requests.get('https://money.cnn.com/data/dow30/')
    except ConnectionError as err:
        print(err) 
    search_pattern = re.compile('class="wsod_symbol">(.*?)<\/a>.*?<span.*?">(.*?)<\/span>.*?\n.*?class="wsod_stream">(.*?)<\/span>')
    dji_list_in_text = re.findall(search_pattern, r.text)
    dji_list = []
    for item in dji_list_in_text:
        dji_list.append([item[0], item[1], float(item[2])])
    return dji_list
 
dji_list = retrieve_dji_list()
djidf = pd.DataFrame(dji_list)
print(djidf)

       0                        1       2
0    MMM                       3M  187.95
1    AXP         American Express  116.59
2   AAPL                    Apple  201.10
3     BA                   Boeing  376.39
4    CAT              Caterpillar  138.43
5    CVX                  Chevron  121.24
6   CSCO                    Cisco   55.82
7     KO                Coca-Cola   48.98
8    DIS                   Disney  137.03
9    DOW             Dow Chemical   56.16
10   XOM              Exxon Mobil   80.09
11    GS            Goldman Sachs  205.25
12    HD               Home Depot  201.75
13   IBM                      IBM  139.45
14  INTC                    Intel   51.27
15   JNJ        Johnson & Johnson  140.37
16   JPM           JPMorgan Chase  116.03
17   MCD               McDonald's  197.51
18   MRK                    Merck   77.45
19  MSFT                Microsoft  129.88
20   NKE                     Nike   87.68
21   PFE                   Pfizer   40.52
22    PG         Procter & Gamble 

In [3]:
# 考察点： 整理数据, 改变列名, index
# 请把列名分别改为：['code','name','lasttrade']（第一列为 code，第二列为 name，第三列为lasttrade
# ————————请写下你的代码——————（此处也可改为选择题）


print(djidf)
'''
输出示例：
    code                     name  lasttrade
1    MMM                       3M     187.95
2    AXP         American Express     116.59
3   AAPL                    Apple     201.10
4     BA                   Boeing     376.39
'''

'''参考答案如下：
cols=['code','name','lasttrade']
djidf.columns=cols # 改变列名
djidf.index=range(1,len(djidf)+1)
print(djidf)
'''

    code                     name  lasttrade
1    MMM                       3M     187.95
2    AXP         American Express     116.59
3   AAPL                    Apple     201.10
4     BA                   Boeing     376.39
5    CAT              Caterpillar     138.43
6    CVX                  Chevron     121.24
7   CSCO                    Cisco      55.82
8     KO                Coca-Cola      48.98
9    DIS                   Disney     137.03
10   DOW             Dow Chemical      56.16
11   XOM              Exxon Mobil      80.09
12    GS            Goldman Sachs     205.25
13    HD               Home Depot     201.75
14   IBM                      IBM     139.45
15  INTC                    Intel      51.27
16   JNJ        Johnson & Johnson     140.37
17   JPM           JPMorgan Chase     116.03
18   MCD               McDonald's     197.51
19   MRK                    Merck      77.45
20  MSFT                Microsoft     129.88
21   NKE                     Nike      87.68
22   PFE  

In [4]:
# 请输出 code 列
# ————————请写下你的代码——————（此处也可改为选择题）


'''
输出示例：
1      MMM
2      AXP
3     AAPL
4       BA
5      CAT
'''

'''
参考答案如下：
djidf.code   # 获取code列+index
djidf['code']  # 获取code列 , 两者同功能
'''

1      MMM
2      AXP
3     AAPL
4       BA
5      CAT
6      CVX
7     CSCO
8       KO
9      DIS
10     DOW
11     XOM
12      GS
13      HD
14     IBM
15    INTC
16     JNJ
17     JPM
18     MCD
19     MRK
20    MSFT
21     NKE
22     PFE
23      PG
24     TRV
25     UTX
26     UNH
27      VZ
28       V
29     WMT
30     WBA
Name: code, dtype: object

In [6]:
# 请输出 2-5 行，['code','lasttrade']这 2 列数据
# ————————请写下你的代码——————（此处也可改为选择题）


'''
输出示例：

	code	lasttrade
2	AXP	116.59
3	AAPL	201.10
4	BA	376.39
5	CAT	138.43

'''

'''参考答案如下：
djidf.loc[2:5,['code','lasttrade']] #1-5行, loc表示标签index
'''

,code,lasttrade
2,AXP,116.59
3,AAPL,201.10
4,BA,376.39
5,CAT,138.43


In [7]:
# 简单的数据筛选: 请输出股价大于180的公司名
# ————————请写下你的代码——————（此处也可改为选择题）

'''
输出示例：

1                3M
3             Apple
4            Boeing
12    Goldman Sachs
13       Home Depot
18       McDonald's
26     UnitedHealth
Name: name, dtype: object
'''

'''参考答案如下：
djidf[djidf.lasttrade>=180].name
'''

1                3M
3             Apple
4            Boeing
12    Goldman Sachs
13       Home Depot
18       McDonald's
26     UnitedHealth
Name: name, dtype: object

In [8]:
# python数据分析测试。
# 测试开始前，请运行该代码，初始化程序
####  第二大题 ####
# 该代码将会输出AXP公司过去一年的股价数据获取，并保存为 quotesdf ，这是一个 DataFrame 格式数据。第二大题测试从这里开始
# 注意，该代码对时间进行处理, 将时间转为'%Y-%m-%d'的格式, 并且将这个时间作为一个list 成为quotesdf的index.

import requests
import re
import json
import pandas as pd
from datetime import  date
def retrieve_quotes_historical(stock_code):
    quotes = []
    url = 'https://finance.yahoo.com/quote/%s/history?p=%s' % (stock_code, stock_code)
    try:
        r = requests.get(url)
    except ConnectionError as err:
        print(err)
    m = re.findall('"HistoricalPriceStore":{"prices":(.*?),"isPending"', r.text)
    if m:
        quotes = json.loads(m[0])       # m = ['[{...},{...},...]'] 
        quotes = quotes[::-1]         # 原先数据为date最新的在最前面
    return  [item for item in quotes if not 'type' in item]
quotes = retrieve_quotes_historical('AXP')
list1=[]
for i in range(len(quotes)):
    x=date.fromtimestamp(quotes[i]['date'])
    y=date.strftime(x,'%Y-%m-%d')
    list1.append(y)
quotesdf_ori=pd.DataFrame(quotes,index=list1)    
quotesdf_m = quotesdf_ori.drop(['adjclose'], axis = 1)  #删除adjclose列
quotesdf=quotesdf_m.drop(['date'],axis=1)
print(quotesdf)

                 close        high         low        open   volume
2018-04-30   98.750000  100.370003   98.730003  100.000000  2567000
2018-05-01   98.580002   98.820000   97.290001   98.820000  2915200
2018-05-02   97.629997   99.370003   97.550003   98.059998  3542000
2018-05-03   97.139999   97.320000   95.510002   97.169998  3572300
2018-05-04   98.349998   98.949997   96.040001   96.349998  2363200
2018-05-07   98.580002   98.989998   98.290001   98.529999  2152100
2018-05-08   99.739998  100.370003   98.389999   98.779999  3199100
2018-05-09  100.500000  100.970001   99.489998  100.220001  2694300
2018-05-10  101.379997  101.870003  100.220001  100.709999  2847600
2018-05-11  101.419998  102.680000  101.269997  101.489998  2522100
2018-05-14  101.290001  102.150002  101.110001  101.690002  2234800
2018-05-15  100.739998  101.230003   99.970001  100.820000  3442300
2018-05-16  100.589996  101.239998  100.230003  100.279999  2135200
2018-05-17  101.220001  101.639999  100.300003  

In [20]:
# 简单的数据筛选: 请输出19 年 3 月 22 日至 3 月 31日，收盘价（close）大于 109 的股价数据
# ————————请写下你的代码——————（此处也可改为选择题）


'''
输出示例：


	close	high	low	open	volume
2019-03-22	109.459999	111.320000	108.989998	111.290001	3737400
2019-03-25	109.040001	110.070000	108.190002	109.279999	3596200
2019-03-26	109.500000	110.400002	109.050003	109.989998	2844500
2019-03-28	109.160004	109.879997	108.489998	109.419998	2693300
2019-03-29	109.300003	110.389999	109.139999	110.199997	4891000


'''

'''参考答案如下：
quotesdf[(quotesdf.index>='2019-03-22') & (quotesdf.index<='2019-03-31')& 
(quotesdf.close>=109)]
'''


,close,high,low,open,volume
2019-03-22,109.459999,111.320000,108.989998,111.290001,3737400
2019-03-25,109.040001,110.070000,108.190002,109.279999,3596200
2019-03-26,109.500000,110.400002,109.050003,109.989998,2844500
2019-03-28,109.160004,109.879997,108.489998,109.419998,2693300
2019-03-29,109.300003,110.389999,109.139999,110.199997,4891000


In [21]:

# 简单的数据统计: 请统计AXP股价涨跌的天数 (close>open)
# ————————请写下你的代码——————（此处也可改为选择题）


'''
输出示例：


252

'''

'''参考答案如下：
len(quotesdf.close>quotesdf.open) 
'''





252

In [22]:

# 简单的数据统计: 请使用describe函数, 得到基本统计信息（超级强大，代码一行）
# ————————请写下你的代码——————（此处也可改为选择题）


'''
输出示例：
	close	high	low	open	volume
count	252.000000	252.000000	252.000000	252.000000	2.520000e+02
mean	104.331210	105.137738	103.487580	104.315754	3.496352e+06
std	5.118420	5.051194	5.171894	5.134740	1.344630e+06
min	89.500000	91.550003	89.050003	89.849998	5.894990e+05
25%	100.495001	101.137501	99.612501	100.215000	2.598775e+06
50%	104.154999	104.934998	103.215000	104.049999	3.232900e+06
75%	108.250000	109.222502	107.342497	108.212498	4.104825e+06
max	117.599998	117.730003	116.550003	117.239998	1.100530e+07
'''

'''参考答案如下：
quotesdf.describe()
'''

,close,high,low,open,volume
count,252.000000,252.000000,252.000000,252.000000,2.520000e+02
mean,104.331210,105.137738,103.487580,104.315754,3.496352e+06
std,5.118420,5.051194,5.171894,5.134740,1.344630e+06
min,89.500000,91.550003,89.050003,89.849998,5.894990e+05
25%,100.495001,101.137501,99.612501,100.215000,2.598775e+06
50%,104.154999,104.934998,103.215000,104.049999,3.232900e+06
75%,108.250000,109.222502,107.342497,108.212498,4.104825e+06
max,117.599998,117.730003,116.550003,117.239998,1.100530e+07


In [23]:

# 简单的数据统计: 请统计2019 年一月的交易日天数
# ————————请写下你的代码——————（此处也可改为选择题）


'''
输出示例：
21
'''

'''参考答案如下：
t=quotesdf[(quotesdf.index>='2019-01-01') & (quotesdf.index<'2019-02-01')]
len(t)  #21
'''


21